In [1]:
from common_import import *

In [2]:
# load model
checkpoint_name = 'MB_train_h36m_gt_cam_no_factor_input_from_canonical_pcl_original'
args, opts = get_opt_args_from_model_name(checkpoint_name, verbose=False)
model_pos, chk_filename, checkpoint = load_model(opts, args)
# load dataset
subset = 'H36M-GT-CAM_NO_FACTOR-INPUT_FROM_CANONICAL_PCL_ORIGINAL'
if 'PCL' in subset: args.fix_orientation_pred = True # load dataset
args.subset_list = [subset]
train_loader, test_loader, _, _, datareader = load_dataset(args, use_new_datareader=True)
e1, e2, results_all, inputs_all, gts_all, total_result_dict = evaluate(args, model_pos, test_loader, datareader, checkpoint, only_one_batch=False)

MB_train_h36m_gt_cam_no_factor_input_from_canonical_pcl_original


Loading checkpoint checkpoint/pose3d/MB_train_h36m_gt_cam_no_factor_input_from_canonical_pcl_original/best_epoch.bin
Loading dataset...
H36M-GT-CAM_NO_FACTOR-INPUT_FROM_CANONICAL_PCL_ORIGINAL
[overwrite: False] ==> Loading H36M source_list...
[overwrite: False] ==> Loading H36M cam_param...
[overwrite: False] ==> Loading H36M cam_3d...
[overwrite: False] ==> Loading H36M cam_3d_canonical_pcl_original...
[overwrite: False] ==> Loading H36M img_2d_canonical_pcl_original...
INFO: Testing
No epoch information in the checkpoint


100%|██████████| 133/133 [00:46<00:00,  2.84it/s]


No eval_keypoint. Use part list
Part: whole
Protocol #1 Error (MPJPE): 23.633188628847385 mm
Protocol #2 Error (P-MPJPE): 18.086285298231353 mm
----------------------------------------


In [5]:
batch_num = 0
pred_3d = results_all[batch_num].copy()
input_2d = inputs_all[batch_num].copy()
gt_3d = gts_all[batch_num].copy()

In [6]:
pwd

'/home/hrai/codes/PerspectiveCropLayers/src'

In [ ]:
## check original train data
frame_num = 500
configs = {
    'ax_3d': {'type': '3d', 'loc': 221, 'mode': 'cam', 'zlim': [0, 6]},
    'ax_3d_top': {'type': '3d', 'loc': 222, 'mode': 'cam_top', 'zlim': [0, 6]},
    'ax_2d': {'type': '2d', 'loc': 223, 'W': W, 'H': H},
    'ax_2d_virt': {'type': '2d', 'loc': 224, 'W': 1, 'H': 1},
}
fig, axs = generate_axes(2, configs, fig_title='')
plot_pose_setting = [
    ('ax_3d', cam_3d,                               {'color': '', 'label': 'original'}),
    ('ax_3d', cam_3d_canonical_revolute,            {'color': 'k', 'label': 'canonical_revolute'}),
    ('ax_3d_top', cam_3d,                           {'color': '', 'label': 'original'}),
    ('ax_3d_top', cam_3d_canonical_revolute,        {'color': 'k', 'label': 'canonical_revolute'}),
    ('ax_2d', img_2d,                               {'color': '', 'label': 'original'}),
    ('ax_2d', img_2d_canonical_pcl,                 {'color': 'r', 'label': 'pcl_with_K'}),
    ('ax_2d', img_2d_canonical_pcl_with_Rz,         {'color': 'b', 'label': 'pcl_with_K_Rz', 'linestyle': '--'}),
    ('ax_2d_virt', img_2d_canonical_pcl_original,   {'color': '', 'label': 'pcl_original'}),
]
general_plot_func(axs, configs, plot_pose_setting, frame_num)
plt.sca(axs['ax_3d'])
# camera
camera = Camera(origin=np.array([0, 0, 0]),
                calib_mat=intrinsic,
                cam_default_R=np.eye(3),
                IMAGE_HEIGHT=H,
                IMAGE_WIDTH=W,
                cam_name='cam_frame')
camera.cam_frame.draw3d(show_name=False)
plt.show()